In [1]:
# !pip install findspark

In [1]:
import findspark
findspark.init('/usr/local/spark')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession\
    .builder\
    .master("spark://master:7077")\
    .config("spark.jars", "spark_dependency_jars//ojdbc11.jar")\
    .appName("Spark to Oracle Processing")\
    .getOrCreate()

24/04/16 23:34:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark

In [5]:
driver="oracle.jdbc.driver.OracleDriver"
url = "jdbc:oracle:thin:@//192.168.78.100:1521/palani6"
user="hduser"
passwd="hadoop"
table="bike_rental_station_events"

In [6]:
table_df=spark.read.format('jdbc')\
        .option("driver", "oracle.jdbc.driver.OracleDriver")\
        .option("url", url) \
        .option("user", user) \
        .option("password", passwd) \
        .option("dbtable", table)\
        .load()

In [7]:
table_df.show()

+-------------+-------------+--------------------+-------+------------+----------+
|          ENV|           ID|               EVENT|STATION|       Q_LEN|BIKE_COUNT|
+-------------+-------------+--------------------+-------+------------+----------+
| 0.3869722638| 1.0000000000|has arrived at st...|      A|0.0000000000|   5 bikes|
| 1.1336081898| 2.0000000000|has arrived at st...|      A|0.0000000000|   4 bikes|
| 1.5565293000| 4.0000000000|has arrived at st...|      A|0.0000000000|   2 bikes|
| 1.8669777319| 6.0000000000|has arrived at st...|      B|0.0000000000|   5 bikes|
| 1.5249224939| 3.0000000000|has arrived at st...|      A|0.0000000000|   3 bikes|
| 1.8316965160| 5.0000000000|has arrived at st...|      A|0.0000000000|   1 bikes|
| 4.7438040177|13.0000000000|has arrived at st...|      B|0.0000000000|   5 bikes|
| 4.1576102145| 7.0000000000|has arrived at st...|      A|0.0000000000|   2 bikes|
| 5.0786745783|15.0000000000|has arrived at st...|      B|0.0000000000|   4 bikes|
| 4.

In [20]:
csv_df = spark.read.option("delimiter", ",").option("header", True).csv("/user/yogov/event_generator.csv")
csv_df.show()

+-------------------+---+--------------------+-------+-----+----------+
|                env| id|               event|station|q_len|bike_count|
+-------------------+---+--------------------+-------+-----+----------+
|0.38697226384132344|  1|has arrived at st...|      A|    0|   5 bikes|
| 1.1336081898341615|  2|has arrived at st...|      A|    0|   4 bikes|
|  1.556529300023573|  4|has arrived at st...|      A|    0|   2 bikes|
| 1.8669777318902117|  6|has arrived at st...|      B|    0|   5 bikes|
| 1.5249224939113988|  3|has arrived at st...|      A|    0|   3 bikes|
| 1.8316965159790086|  5|has arrived at st...|      A|    0|   1 bikes|
| 4.7438040177186735| 13|has arrived at st...|      B|    0|   5 bikes|
| 4.1576102145237215|  7|has arrived at st...|      A|    0|   2 bikes|
|  5.078674578267292| 15|has arrived at st...|      B|    0|   4 bikes|
|  4.286977457434506|  8|has arrived at st...|      A|    0|   1 bikes|
|  4.376462840745697|  9|has arrived at st...|      A|    0|   0

In [21]:
csv_df.count()

410

In [25]:
csv_df.write\
        .format('jdbc')\
        .option("driver", "oracle.jdbc.OracleDriver")\
        .option("url", url) \
        .option("user", user) \
        .option("password", passwd) \
        .mode("append") \
        .option("CreateTableOptions", '') \
        .option("dbtable", table) \
        .save()